# 🌾 AgriCash Data Pipeline: Scraping & RAG Preparation

### **Technical Discovery Report: AgriCash Data Extraction**

#### **1. Initial Site Reconnaissance (HTML Analysis)**
We began by analyzing the raw HTML response of the `/shop` page. This is the standard entry point for web scraping, but upon inspection, several key architectural details were revealed:
*   **Platform Identification:** The structure of the HTML tags, specific CSS classes (e.g., `.woocommerce-product-gallery`), and script IDs confirmed the site is built on **WordPress** using the **WooCommerce** e-commerce framework.
*   **Multi-Vendor System:** We identified scripts related to **Dokan**, indicating this is a marketplace with multiple sellers, not just a single store.
*   **Complexity:** The HTML contained heavy inline Javascript and nested `<div>` structures. While parsing this with a tool like BeautifulSoup is possible, it is brittle; a simple change in the site’s theme or layout would break the scraper immediately.

#### **2. The Endpoint Discovery (Header Analysis)**
Instead of bruteforcing the HTML, we analyzed the HTTP Headers sent by the server during a standard request. This revealed the "Golden Key" to the data:
*   **The REST API Link:** The server returned a specific header: `link: <https://agricash.app/wp-json/>; rel="https://api.w.org/"`. This explicitly announced that the WordPress REST API is active and accessible to the public.
*   **Server Security:** The headers identified the server as **LiteSpeed**. This is crucial for scraping strategy, as LiteSpeed is aggressive against bots. It confirms that any automated requests must use a valid "User-Agent" string to mimic a real web browser, or the server will reject the connection (403 Forbidden).

#### **3. Locating the Data Source (The Store API)**
Using the knowledge that the API was active, we moved away from the standard WordPress endpoints and located the specialized **WooCommerce Store API** (`/wp-json/wc/store/v1/products`).
*   **Data Structure:** Unlike the HTML, which is designed for human eyes, this endpoint returns data in **JSON format**. This structure is machine-readable, lightweight, and strictly organized.
*   **Rich Data Access:** The API provided immediate access to high-value data points that are often difficult to parse from HTML, including:
    *   **Variable Pricing:** Distinct structures for simple products (fixed price) vs. variable products (price ranges, e.g., "55-65 EGP").
    *   **Stock Status:** Boolean values (True/False) indicating if an item is purchasable, rather than relying on text labels like "Out of Stock."
    *   **Image Assets:** Direct URLs to high-resolution product images without needing to parse `srcsets`.

#### **4. Analyzing the "Instruction Manual" (API Schema)**
By querying the root namespace (`/wp-json/wc/store/v1/`), we retrieved the **API Schema**. This acts as the official documentation for the site's backend, revealing capabilities that are otherwise invisible:
*   **Speed Optimization:** The schema explicitly defined the pagination limits. We found that the `per_page` parameter defaults to 10 but allows a **maximum of 100 items per request**. This discovery reduces the number of HTTP requests needed by 90% compared to standard scraping.
*   **Server-Side Filtering:** We discovered the server supports advanced filtering parameters. Instead of downloading every product and filtering them in our code, we can request exactly what we need directly from the server:
    *   **`stock_status`:** We can request only "instock" items.
    *   **`on_sale`:** We can request only items currently discounted.
    *   **`min_price` / `max_price`:** We can filter by budget range.
    *   **`search`:** We can perform text searches directly via the API.
*   **Additional Endpoints:** The schema revealed other valuable data pools, including **Categories**, **Brands**, and **Product Reviews**, which can be accessed via their own dedicated URLs.

### **Conclusion**
We have successfully transitioned from a fragile "Screen Scraping" approach (parsing HTML) to a robust "API Consumption" strategy. By leveraging the site's own internal API and adhering to the rules defined in their schema, we can extract data faster, more reliably, and with zero risk of the scraper breaking due to visual website changes.

In [22]:
import requests
import json

# 1. Setup - We use the 'Store API' as discovered
url = "https://agricash.app/wp-json/wc/store/v1/products"

# 2. Headers - Essential to bypass LiteSpeed security
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
    "Referer": "https://agricash.app/shop/"
}

# 3. Fetch just 1 product to inspect structure
params = {
    "per_page": 1,
    "page": 1
}

response = requests.get(url, headers=headers, params=params)


In [23]:
response.json()

[{'id': 7759,
  'name': 'نيو روش كيلر',
  'slug': 'new-rouch-killer',
  'parent': 0,
  'type': 'variable',
  'variation': '',
  'permalink': 'https://agricash.app/product/new-rouch-killer/',
  'sku': '',
  'short_description': '<p>نيو روش كيلر</p>\n<p>يستخدم لمكافحة الحشرات الزاحفة</p>',
  'description': '<p>نيو روش كيلر</p>\n<p>يستخدم لمكافحة الحشرات الزاحفة</p>',
  'on_sale': False,
  'prices': {'price': '55',
   'regular_price': '55',
   'sale_price': '55',
   'price_range': {'min_amount': '55', 'max_amount': '65'},
   'currency_code': 'EGP',
   'currency_symbol': 'EGP',
   'currency_minor_unit': 0,
   'currency_decimal_separator': ',',
   'currency_thousand_separator': '.',
   'currency_prefix': '',
   'currency_suffix': ' EGP'},
  'price_html': '<span class="woocommerce-Price-amount amount" aria-hidden="true">55&nbsp;<span class="woocommerce-Price-currencySymbol">EGP</span></span> <span aria-hidden="true"> – </span> <span class="woocommerce-Price-amount amount" aria-hidden="true">

In [24]:
data = response.json()

# the useful metadata
if data:
    product = data[0]

    print("=== PRODUCT KEYS AVAILABLE ===")
    print(product.keys())

    print("\n=== CRITICAL FIELDS INSPECTION ===")
    print(f"Name: {product.get('name')}")
    print(f"ID: {product.get('id')}")
    print(f"price: {product.get('prices').get("price")}")
    print(f"price range: {product.get('prices').get("price_range")}")
    print(f"Stock Status: {product.get('is_in_stock')}")
    print(f"link: {product.get('permalink')}")

    print("\n--- Categories (Hierarchy Check) ---")
    # This is crucial for RAG. We need to see if it gives us a path or just a list.
    print(json.dumps(product.get('categories'), indent=2, ensure_ascii=False))

    print("\n--- Description (HTML Check) ---")
    # We need to see how much cleaning this needs
    print(product.get('description')[:200]) # Printing first 200 chars only
    print(product.get('short_description')[:200]) # Printing first 200 chars only
else:
    print("No products found.")

=== PRODUCT KEYS AVAILABLE ===
dict_keys(['id', 'name', 'slug', 'parent', 'type', 'variation', 'permalink', 'sku', 'short_description', 'description', 'on_sale', 'prices', 'price_html', 'average_rating', 'review_count', 'images', 'categories', 'tags', 'brands', 'attributes', 'variations', 'grouped_products', 'has_options', 'is_purchasable', 'is_in_stock', 'is_on_backorder', 'low_stock_remaining', 'stock_availability', 'sold_individually', 'add_to_cart', 'extensions'])

=== CRITICAL FIELDS INSPECTION ===
Name: نيو روش كيلر
ID: 7759
price: 55
price range: {'min_amount': '55', 'max_amount': '65'}
Stock Status: True
link: https://agricash.app/product/new-rouch-killer/

--- Categories (Hierarchy Check) ---
[
  {
    "id": 40,
    "name": "مبيدات الصحة العامة",
    "slug": "public-health-pesticides",
    "link": "https://agricash.app/product-category/plant/pesticides/public-health-pesticides/"
  }
]

--- Description (HTML Check) ---
<p>نيو روش كيلر</p>
<p>يستخدم لمكافحة الحشرات الزاحفة</p>
<

## Step 2: Data Cleaning & Formatting Strategy


In [29]:
from bs4 import BeautifulSoup
import json

# 1. Helper function to clean HTML (p tags, etc)
def clean_html(html_content):
    if not html_content:
        return ""
    # use 'html.parser' so you don't need extra libraries
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text(separator="\n").strip()

# 2. Helper to handle variable prices
def get_price_string(prices_data):
    # Check if there is a range (min/max are different)
    price_range = prices_data.get('price_range', {})
    if price_range and price_range.get('min_amount') != price_range.get('max_amount'):
        return f"{price_range.get('min_amount')} - {price_range.get('max_amount')} {prices_data.get('currency_code')}"

    # Fallback to standard price
    return f"{prices_data.get('price')} {prices_data.get('currency_code')}"

# 3. Process the single product we fetched earlier
# (Assuming 'product' variable still holds the data from your previous step)

fine_grained_item = {
    "id": product.get("id"),
    "name": product.get("name"),
    "url": product.get("permalink"),
    "in_stock": product.get("is_in_stock", False),

    # Cleaned Price
    "price": product.get('prices').get("price"),
    "price_range": get_price_string(product.get('prices')),

    # Cleaned Text
    "short_description": clean_html(product.get("short_description")),
    "description": clean_html(product.get("description")),

    # Categories (We currently only have the immediate ones, hierarchy comes later)
    "categories": [cat.get("name") for cat in product.get("categories", [])],
}

# Print the JSON result
print(json.dumps(fine_grained_item, indent=4, ensure_ascii=False))

{
    "id": 7759,
    "name": "نيو روش كيلر",
    "url": "https://agricash.app/product/new-rouch-killer/",
    "in_stock": true,
    "price": "55",
    "price_range": "55 - 65 EGP",
    "short_description": "نيو روش كيلر\n\n\nيستخدم لمكافحة الحشرات الزاحفة",
    "description": "نيو روش كيلر\n\n\nيستخدم لمكافحة الحشرات الزاحفة",
    "categories": [
        "مبيدات الصحة العامة"
    ]
}


# Hierarchy Map

The **API Schema** gave us the blueprint for this. Specifically, it showed us the endpoint `/wc/store/v1/products/categories`

To get the full hierarchy (e.g., **"Plants > Pesticides > Public Health"** instead of just "Public Health"), we need to:
1.  Download **all** categories first.
2.  Build a "Map" that links every Category ID to its Parent ID.
3.  Write a small function to walk up that chain.


### Expected Output
If the logic works, instead of just "مبيدات الصحة العامة" (Public Health Pesticides), you should see something like:
`مستلزمات الإنتاج النباتي > المبيدات الزراعية > مبيدات الصحة العامة`
*(Plant Production Supplies > Agricultural Pesticides > Public Health Pesticides)*

This is **Gold** for an LLM because it adds context ("This item is related to Plants").


In [32]:
# 1. Setup based on Schema
cat_url = "https://agricash.app/wp-json/wc/store/v1/products/categories"

# 2. Fetch ALL Categories (Looping because max per_page is 100)
all_categories = []
page = 1

print("Fetching Category Tree...")

while True:
    params = {"per_page": 100, "page": page} # Schema max limit
    response = requests.get(cat_url, headers=headers, params=params)
    data = response.json()

    if not data:
        break

    all_categories.extend(data)
    print(f"Fetched page {page} ({len(data)} categories)")
    page += 1

Fetching Category Tree...
Fetched page 1 (44 categories)


In [45]:
all_categories[0]

{'id': 65,
 'name': 'أحماض أمينية',
 'slug': '%d8%a3%d8%ad%d9%85%d8%a7%d8%b6-%d8%a3%d9%85%d9%8a%d9%86%d9%8a%d8%a9',
 'description': '',
 'parent': 30,
 'count': 17,
 'image': {'id': 4571,
  'src': 'https://i0.wp.com/agricash.app/wp-content/uploads/2024/02/%D8%A7%D8%AD%D9%85%D8%A7%D8%B6-%D8%A7%D9%85%D9%8A%D9%86%D9%8A%D8%A9.png?fit=2048%2C2048&ssl=1',
  'thumbnail': 'https://i0.wp.com/agricash.app/wp-content/uploads/2024/02/%D8%A7%D8%AD%D9%85%D8%A7%D8%B6-%D8%A7%D9%85%D9%8A%D9%86%D9%8A%D8%A9.png?fit=300%2C300&ssl=1',
  'srcset': '',
  'sizes': '(max-width: 2048px) 100vw, 2048px',
  'name': 'احماض امينية',
  'alt': ''},
 'review_count': 0,
 'permalink': 'https://agricash.app/product-category/plant/fertilizers/%d8%a3%d8%ad%d9%85%d8%a7%d8%b6-%d8%a3%d9%85%d9%8a%d9%86%d9%8a%d8%a9/'}

In [46]:
# 3. Create a Lookup Map for O(1) speed
# We map ID -> {Name, Parent_ID}
category_map = {
    cat['id']: {'name': cat['name'], 'parent': cat['parent'], "permalink": cat["permalink"]}
    for cat in all_categories
}

print(f"\nTotal Categories Mapped: {len(category_map)}")


Total Categories Mapped: 44


In [47]:
for key, value in list(category_map.items())[:5]:
    print(f"Key: {key}, Name: {value['name']}, Parent: {value['parent']}")

Key: 65, Name: أحماض أمينية, Parent: 30
Key: 105, Name: أسمدة بوتاسية, Parent: 30
Key: 49, Name: أسمدة-NPK, Parent: 30
Key: 69, Name: أنظمة الري الحديث, Parent: 100
Key: 35, Name: أنظمة الطاقة الشمسية, Parent: 0


so now we have every cat and it's parent we just need to loop over them untill we git to the first parent

In [48]:

# --- FUNCTION 1: Get Ordered List of Objects ---
def get_category_path_objects(cat_id, cat_map):
    """
    Returns a list of category objects from Root -> Leaf.
    Example: [{'id': 15, 'name': 'Plant', ...}, {'id': 40, 'name': 'Pesticide', ...}]
    """
    if cat_id not in cat_map:
        return []

    path = []
    current_id = cat_id

    # Traverse up the tree
    while current_id != 0 and current_id in cat_map:
        cat_obj = cat_map[current_id]

        path.append(cat_obj)
        current_id = cat_obj.get("parent") # Move up

    # Reverse the list so it goes [Grandparent, Parent, Child]
    return path[::-1]

# --- FUNCTION 2: Get Human Readable String ---
def get_category_path_string(path_objects):
    """
    Takes the output of Function 1 and returns a string.
    """
    if not path_objects:
        return "Unknown"

    # Extract just the names and join them
    names = [node['name'] for node in path_objects]
    return " > ".join(names)

# --- TEST IT ---
test_id = 69 # Or 40

# 1. Get the Data Structure
path_data = get_category_path_objects(test_id, category_map)

# 2. Get the String
path_text = get_category_path_string(path_data)

print(f"--- 1. Metadata List (ID: {test_id}) ---")
print(json.dumps(path_data, indent=2, ensure_ascii=False))

print(f"\n--- 2. Human Readable String ---")
print(path_text)

--- 1. Metadata List (ID: 69) ---
[
  {
    "name": "مستلزمات الإنتاج النباتي",
    "parent": 0,
    "permalink": "https://agricash.app/product-category/plant/"
  },
  {
    "name": "أنظمة الري الحديث",
    "parent": 100,
    "permalink": "https://agricash.app/product-category/plant/irrigation/"
  }
]

--- 2. Human Readable String ---
مستلزمات الإنتاج النباتي > أنظمة الري الحديث


### Why this split is better for RAG:
1.  **Function 1 (List):** You pass this to the **Metadata** of your vector (in Pinecone/ChromaDB). This allows you to filter later. (e.g., *"Give me only items where `parent` is `40`"*).
2.  **Function 2 (String):** You pass this to the **Embedding Text** so the LLM reads it naturally.

We have the **Ingredients**:
1.  **The API Connection** (Step 1)
2.  **The HTML & Price Cleaners** (Step 2)
3.  **The Category Hierarchy Map** (Step 3)

### Step 4: Building the "Processor" Function
Before we run the big loop to download everything, we need a single function that combines all previous steps.

This function will take a **Raw Product** (from the API) and transform it into a **RAG-Ready Dictionary**. This is the most critical part of your pipeline because this is exactly what your LLM will "see."


In [50]:
def process_product_for_rag(raw_product, category_map):
    """
    Takes a single Raw Product from the API and transforms it into
    a clean, hierarchical, RAG-ready dictionary with Arabic Context.
    """

    # 1. Calculate Best Category Hierarchy (Longest Path)
    raw_cats = raw_product.get('categories', [])
    best_path_objs = []
    best_path_str = "غير مصنف" # Default if no category

    if raw_cats:
        paths = []
        for c in raw_cats:
            p_objs = get_category_path_objects(c['id'], category_map)
            paths.append(p_objs)

        if paths:
            best_path_objs = max(paths, key=len)
            best_path_str = get_category_path_string(best_path_objs)

    # 2. Clean Descriptions
    short_desc = clean_html(raw_product.get("short_description", ""))
    full_desc = clean_html(raw_product.get("description", ""))

    # Combine Short + Long as requested
    combined_description = f"{short_desc}\n{full_desc}".strip()

    # 3. Handle Price (Single Value, not Range)
    # The API returns 'price' as the current active price (sale or regular)
    current_price = raw_product.get('prices', {}).get('price', 'غير معلوم السعر')

    # 4. Construct Final Object
    fine_grained_item = {
        # --- Metadata (For filtering in Vector DB) ---
        "id": raw_product.get("id"),
        "name": raw_product.get("name"),
        "url": raw_product.get("permalink"),
        "in_stock": raw_product.get("is_in_stock", False),

        # Category Metadata
        "category_hierarchy": best_path_objs,

        # --- Content (For LLM Generation) ---
        "price": current_price,
        "short_description": short_desc,
        "description": full_desc,
        "combined_description": combined_description,
        "category_text": best_path_str,

        # --- The Golden Field: RAG Context (Egyptian Arabic) ---
        "rag_context_text": f"""
اسم المنتج: {raw_product.get('name')}
التصنيف: {best_path_str}
السعر: {current_price} جنيه
الحالة: {"متاح" if raw_product.get('is_in_stock') else "غير متاح"}
الوصف:
{combined_description}
""".strip()
    }

    return fine_grained_item


processed_item = process_product_for_rag(product, category_map)
print(json.dumps(processed_item, indent=4, ensure_ascii=False))


{
    "id": 7759,
    "name": "نيو روش كيلر",
    "url": "https://agricash.app/product/new-rouch-killer/",
    "in_stock": true,
    "category_hierarchy": [
        {
            "name": "مستلزمات الإنتاج النباتي",
            "parent": 0,
            "permalink": "https://agricash.app/product-category/plant/"
        },
        {
            "name": "المبيدات",
            "parent": 100,
            "permalink": "https://agricash.app/product-category/plant/pesticides/"
        },
        {
            "name": "مبيدات الصحة العامة",
            "parent": 29,
            "permalink": "https://agricash.app/product-category/plant/pesticides/public-health-pesticides/"
        }
    ],
    "price": "55",
    "short_description": "نيو روش كيلر\n\n\nيستخدم لمكافحة الحشرات الزاحفة",
    "description": "نيو روش كيلر\n\n\nيستخدم لمكافحة الحشرات الزاحفة",
    "combined_description": "نيو روش كيلر\n\n\nيستخدم لمكافحة الحشرات الزاحفة\nنيو روش كيلر\n\n\nيستخدم لمكافحة الحشرات الزاحفة",
    "category


1.  **Nested Data:** Your `category_hierarchy` is a **list of objects** (`[{'id': 40, 'name': '...'}, ...]`).
    *   **CSV** creates a mess here. It turns that list into a string. To use it later, you have to write code to "un-stringify" it.
    *   **JSONL** keeps it as a real list/object. You load it, and it's ready to use.
2.  **Multiline Arabic Text:** Your `rag_context_text` has newlines (`\n`) and Arabic characters.
    *   **CSV** often breaks or displays garbage characters in Excel when handling multiline Arabic text unless encoding is handled perfectly.
    *   **JSONL** handles Unicode and newlines natively without issues.
3.  **Crash Safety:**
    *   **JSONL** writes line-by-line. If your internet cuts out after 500 items, you still have those 500 items saved.
    *   **Standard JSON** requires a closing `]` at the very end. If the script crashes, the whole file is often corrupt.

### Plan

1.  **Preparation:** Ensure `category_map` is built (from Step 3).
2.  **File Setup:** Open `agricash_dataset.jsonl` for writing (`w` mode).
3.  **The Loop:**
    *   Fetch **100 products** per page.
    *   **Keep Out-of-Stock items** (so the LLM knows they exist but are currently unavailable).
    *   Process each item using `process_product_for_rag`.
    *   **Immediately write** the processed line to the JSONL file.
4.  **Completion:** Print the total count.



In [51]:
import requests
import json
import time

# --- CONFIGURATION ---
URL = "https://agricash.app/wp-json/wc/store/v1/products"
OUTPUT_FILE = "agricash_rag_dataset.jsonl"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
    "Referer": "https://agricash.app/shop/"
}

# Ensure category_map exists from previous steps
if 'category_map' not in locals():
    raise ValueError("⚠️ Please run Step 3 first to generate the 'category_map'!")

print(f"🚀 Starting Scraper...")
print(f"📂 Output File: {OUTPUT_FILE}")

# Open the file in write mode
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    page = 1
    total_items = 0

    while True:
        print(f"⏳ Fetching Page {page} (100 items)...", end=" ")

        try:
            params = {
                "per_page": 100,  # Maximize throughput
                "page": page
            }

            response = requests.get(URL, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"\n❌ Error: Status Code {response.status_code}")
                break

            data = response.json()

            # STOP Condition: API returns empty list
            if not data:
                print("✅ Done. No more products.")
                break

            # Process & Write immediately
            for raw_product in data:
                # 1. Process
                processed_item = process_product_for_rag(raw_product, category_map)

                # 2. Write as JSON Line
                json_line = json.dumps(processed_item, ensure_ascii=False)
                f.write(json_line + "\n")

                total_items += 1

            print(f"-> Saved {len(data)} items.")
            page += 1

            # Polite sleep to prevent blocking
            time.sleep(1)

        except Exception as e:
            print(f"\n❌ Critical Error: {e}")
            break

print(f"\n🎉 Finished! Total Products Scraped: {total_items}")
print(f"📄 Saved to: {OUTPUT_FILE}")

🚀 Starting Scraper...
📂 Output File: agricash_rag_dataset.jsonl
⏳ Fetching Page 1 (100 items)... -> Saved 100 items.
⏳ Fetching Page 2 (100 items)... -> Saved 100 items.
⏳ Fetching Page 3 (100 items)... -> Saved 100 items.
⏳ Fetching Page 4 (100 items)... -> Saved 100 items.
⏳ Fetching Page 5 (100 items)... -> Saved 100 items.
⏳ Fetching Page 6 (100 items)... -> Saved 53 items.
⏳ Fetching Page 7 (100 items)... ✅ Done. No more products.

🎉 Finished! Total Products Scraped: 553
📄 Saved to: agricash_rag_dataset.jsonl


### How to use the output file (`.jsonl`):
When you are building your RAG system (e.g., using LangChain or LlamaIndex), you load the data like this:

```python
# Example: How to load the data back
with open('agricash_rag_dataset.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line)
        # item['rag_context_text'] -> Goes to Embedding
        # item['price'] -> Goes to Metadata
```

first approuch we scraped the loaded html itself it result in less data and more noise but we tested the page and found the api endpoints without any limits so we took the oppourtunity